<a href="https://colab.research.google.com/github/stevenct84/Algoritmos-de-Machine-Learning/blob/main/K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [204]:
#Inicio:13/09/2022
#Fin: 2/10/2022
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time

In [ ]:
#Subir el archivo csv
#Seleccionar el dataset (Players.csv)
from google.colab import files
files.upload()

In [ ]:
#cargar el dataset usando pandas
players=pd.read_csv('Players.csv',sep=';')
players

In [ ]:
#Seleccionar las columnas que se van a usar.
#Eliminar los datos vacios con dropna
features=["overall", "potential", "wage_eur", "value_eur", "age"] 
players=players.dropna(subset=features)
dataPlayer=players[features].copy()
dataPlayer

In [ ]:
#mostrar algunas estadisticas sobre el dataframe
dataPlayer.describe()

In [264]:
#Crear un nuevo dataframe
#Para crear los datos nuevos según las estadisticas de dataPlayer
data=pd.DataFrame()

In [265]:
#Se crean nuevas columnas en el dataframe
#Los nuevos datos se crean entre 2 intervalos y una cantidad de datos.
data["overall"]=np.random.randint(low=47, high=93, size=(15000,))
data["potential"]=np.random.randint(low=49, high=95, size=(15000,))
data["wage_eur"]=np.random.randint(low=500, high=350000, size=(15000,))
data["value_eur"]=np.random.randint(low=900, high=194000000, size=(15000,))
data["age"]=np.random.randint(low=16, high=43, size=(15000,))

In [ ]:
#Ver las estadisticas del dataframe que se utiliza para el algoritmo
data.describe()

In [ ]:
#Pasos a seguir para realizar el algoritmo de k-means
#Escalar los datos
#Inicializar los centroides de manera random
#(calcular la distancia entre los puntos)
#Actualizar los centroides
#Repetir 3 y 4 hasta que los centroides paren de cambiar


In [267]:
#Escalar los datos
#Hay datos que tiene rangos muy grandes, 
#se piensa cambiar para obtener todo entre 1 a 10
data= ((data-data.min())/(data.max()-data.min()))*9 + 1

In [ ]:
#asignar las variables globales para contar asignacioes y comparaciones 
asignacion, comparacion,linea=0,0,0

#crear los centroides random
#Data: los datos del dataset
#K: la cantidad de clusters que queremos
#Selecciona un dato random de cada columna y lo coloca como float
def random_centroids(data, k):
  global asignacion, linea, comparacion
  asignacion += 1
  linea+=1
  centroids = []
  for i in range(k):
    comparacion+=1
    asignacion+=2
    linea+=3
    centroid = data.apply(lambda x: float(x.sample())) 
    centroids.append(centroid)
  comparacion+=1
  linea+=2
  return pd.concat(centroids, axis=1) #combina todas las series de pandas en un dataframe


#Crear la etiqueta en cada punto de datos
#Calcular la distancia entre los centroides y los datos
#data: todos los datos
#centroids: los centroides actuales del programa
def get_labels(data, centroids):
  global asignacion, linea
  asignacion+=1
  linea+=2
  distances=centroids.apply(lambda x: np.sqrt(((data-x)**2).sum(axis=1)))
  return distances.idxmin(axis=1)

#Se calcula los centroides nuevos
#data: los datos del dataset
#labels: distancias de los puntos al centroide
def new_centroids(data, labels, k):
    global asignacion, linea
    asignacion+=1
    linea+=2
    centroids = data.groupby(labels).apply(lambda x: np.exp(np.log(x).mean())).T
    return centroids


#Funcion principal, llama a las demás creadas
#data:los datos del dataset
#centroid_count: cantidad de grupos que se desean realizar
def kMeans(data,centroid_count):
  inicio = time.time()
  global asignacion, linea, comparacion
  asignacion+=1
  linea+=4
  max_iterations = 100  
  asignacion=+3
  centroids = random_centroids(data, centroid_count)  #asignar de manera aleatoria los centroides
  old_centroids = pd.DataFrame()
  iteration = 1

  while iteration < max_iterations and not centroids.equals(old_centroids):  #verificar si se debe finalizar
      linea+=5
      comparacion+=2
      asignacion+=4
      old_centroids = centroids
      labels = get_labels(data, centroids)
      centroids = new_centroids(data, labels, centroid_count)
      iteration += 1
  linea+=2
  comparacion+=2
  fin = time.time()
  tiempo= fin-inicio
  print("Tiempo: %.4f seconds." %tiempo)
  return centroids

#Correr el algoritmo de K-means
centroids=kMeans(data,4)
print("Asignaciones:")
print(asignacion)
print("Comparacaciones:")
print(comparacion)
print("Lineas ejecutadas:")
print (linea)



In [ ]:
#Revisar la cantidad de integrantes por grupo
labels=get_labels(data, centroids)
data["label"]=labels

uno=0
dos=0
tres=0
cero=0
for i in data["label"]:
  if i==1:
    uno+=1
  elif i==2:
    dos+=1
  elif i==3:
    tres+=1
  else:
    cero+=1
print("Cantidad de datos: " + str(len(labels)))
print("Grupo con etiqueta 1: "+ str(uno))
print("Grupo con etiqueta 2: "+ str(dos))
print("Grupo con etiqueta 3: "+ str(tres))
print("Grupo con etiqueta 0: "+ str(cero))

In [ ]:
#Graficar el resultado
import matplotlib.pyplot as plt
#se agrega al dataset la columna label para etiquetar los resultados
u_labels = np.unique(labels)

#graficar el resultado
for i in u_labels:
    plt.scatter(x=data["overall"][data["label"]==i] ,y= data["potential"][data["label"]==i] , label = i)
plt.scatter(centroids.iloc[0,] , centroids.iloc[1,] , s = 80, color = 'k')
plt.legend()
plt.show()